In [1]:
from importlib import reload
import utils
reload(utils)
import model_class
reload(model_class)

from model_class import *
from utils import *
import torch.nn as nn
import torch

In [2]:
device = 'cpu'
model_file_name = 'model-parameters/model.tar'

hidden_size = 500
encoder_n_layers = 3
decoder_n_layers = 3
dropout = 0.0
attn_model = 'dot'

max_length = 4  # Maximum sentence length to consider

# Default word tokens
#PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
#EOS_token = 2  # End-of-sentence token

In [3]:
# Instantiate vocabulary
voc = Voc()

In [4]:
# Load model parameters and vocabulary
checkpoint = torch.load(model_file_name, map_location=torch.device(device))
encoder_sd = checkpoint['en']
decoder_sd = checkpoint['de']
encoder_optimizer_sd = checkpoint['en_opt']
decoder_optimizer_sd = checkpoint['de_opt']
embedding_sd = checkpoint['embedding']
voc.__dict__ = checkpoint['voc_dict']


In [5]:
# Initialize encoder & decoder models & searcher
embedding = nn.Embedding(voc.num_words, hidden_size)
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
searcher = GreedySearchDecoder(encoder, decoder, device, SOS_token)

# Associate loaded parameters
encoder.load_state_dict(encoder_sd)
decoder.load_state_dict(decoder_sd)

<All keys matched successfully>

In [11]:
get_output(encoder, decoder, searcher, voc, max_length, device, input_sentence='associate external account auditor')

'accountant'

In [12]:
torch.__version__

'1.3.0'